In [1]:
import base64
import pandas as pd
import requests
import requests_negotiate_sspi
from requests_negotiate_sspi import HttpNegotiateAuth
from pandas.io.json import json_normalize
import arcpy
import os
from datetime import datetime
from arcgis.features import GeoAccessor,GeoSeriesAccessor
pd.set_option("display.max_rows",None)
pd.set_option("display.max_columns",None)
pd.set_option("display.max_colwidth",None)

### Setting up variables and requesting a response from the Entities GET function. Then converting the entityTypeList array into a pandas dataframe. The entity TypeList array is a list of dictionaries with key-value pairs. It has entityType,name,description,count,and URL. I take this information and export to excel. This spreadsheet will give us a better understanding of all the entity types we can call within the Studio API. 

In [2]:
# Getting all entity types and descriptions from Studio API
base_url = "http://hoewap609:50231"
connection = "P_PET_SAMP"
repository = "GUYANA_INT"
pagecount = 400

url = f"{base_url}/api/studio/petrel/entities/{connection}/{repository}?entitiesPerPage={pagecount}"
entityTypes = requests.get(url, auth=HttpNegotiateAuth())
entityTypes.raise_for_status()
print(entityTypes.json()) #Response in json format

entityTypes_df = pd.json_normalize(entityTypes.json()['entityTypeList']) #normalizing json formation with pandas json normalize function. This will convert keys to columns and values to data within cells.
entityTypes_df.to_excel(r"H:\Studio_ApI_entityTypes.xlsx") # to excel file
display(entityTypes_df) #displaying dataframe

{'pageNumber': 0, 'entityCount': 237, 'entitiesPerPage': 400, 'pageCount': 1, 'firstPageURL': 'http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT?page=0&entitiesPerPage=400', 'previousPageURL': None, 'nextPageURL': None, 'lastPageURL': 'http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT?page=0&entitiesPerPage=400', 'entityTypeList': [{'entityType': 'DbxCoordinateSystem', 'name': 'SRD', 'description': 'Reference coordinate system associated with lateral and vertical measurements', 'count': 40644, 'entityListURL': 'http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxCoordinateSystem'}, {'entityType': 'DbxReplacementVelocity', 'name': 'Replacement velocity', 'description': 'A place to store Replacement Velocity information. This is assumed velocity between From_SRD and To_SRD.', 'count': 0, 'entityListURL': 'http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxReplacementVelocity'}, {'entityType': 'DbxH

,entityType,name,description,count,entityListURL
0,DbxCoordinateSystem,SRD,Reference coordinate system associated with lateral and vertical measurements,40644,http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxCoordinateSystem
1,DbxReplacementVelocity,Replacement velocity,A place to store Replacement Velocity information. This is assumed velocity between From_SRD and To_SRD.,0,http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxReplacementVelocity
2,DbxHistoryEntry,,History entry associated with Petrel domain object,372135,http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxHistoryEntry
3,DbxCustomAttribute,None,A non-Dbx custom attribute for a DbxObject,5268,http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxCustomAttribute
4,DbxPetrelProject,,Petrel Project Information,4954,http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxPetrelProject
5,DbxPetrelObject,,Petrel Object Information,0,http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxPetrelObject
6,DbxManagedProjectGroup,None,Group Managed Petrel Project,0,http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxManagedProjectGroup
7,DbxManagedProject,None,Managed Petrel Project,0,http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxManagedProject
8,DbxManagedObject,Managed object,Petrel Object Information,0,http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxManagedObject
9,DbxObjectUse,None,Association between Objects,0,http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxObjectUse


#### I noticed the polygons within studio are stored in the entitye DbxPolylineset. We will make a call to the API to see what metadata is avialable to extract. I then convert the entityList array into a dataframe that create a frame that consists of each polygon guid, name, and entityURL. 

In [3]:
entityType = "DbxPolylineSet" # Polygons in studio, A set of polylines.
pagecount = 2000

polylineset_url = f"{base_url}/api/studio/petrel/entities/{connection}/{repository}/{entityType}?entitiesPerPage={pagecount}"
polylineset_req = requests.get(polylineset_url, auth=HttpNegotiateAuth())
polylineset_req.raise_for_status()

df_poly = pd.json_normalize(polylineset_req.json()["entityList"])
display(df_poly)

,guid,name,entityURL
0,00083C8627CE41DE89B6C35381AFF4B6,at_bsclos_non,http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxPolylineSet/00083C8627CE41DE89B6C35381AFF4B6
1,00563B2C96F140A589C0E58B453E6A7C,D_NTG_Low Quality,http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxPolylineSet/00563B2C96F140A589C0E58B453E6A7C
2,005E83FCFA2B4EE995E4F9F901B576ED,Mesquite-441,http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxPolylineSet/005E83FCFA2B4EE995E4F9F901B576ED
3,0066831E14B643149A4FB7505C7CC9C1,Carbonate Slide Block Lead 3,http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxPolylineSet/0066831E14B643149A4FB7505C7CC9C1
4,0092BE4A9D184852885C5B09FB0454B3,LIZ_1P3,http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxPolylineSet/0092BE4A9D184852885C5B09FB0454B3
5,00F0C1F87A234C16A78D99625C544AE4,D_NTG_Medium Quality_1.4,http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxPolylineSet/00F0C1F87A234C16A78D99625C544AE4
6,00F95A61A8B44AE593ECA8D1588E89D1,P2 Clinoforms - 1,http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxPolylineSet/00F95A61A8B44AE593ECA8D1588E89D1
7,0113C64A2E4445F68C2AA6EEA2BB498E,C1 Central,http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxPolylineSet/0113C64A2E4445F68C2AA6EEA2BB498E
8,0123202559704D5AABD1A6AC558F800B,at_salt_outline,http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxPolylineSet/0123202559704D5AABD1A6AC558F800B
9,01275F72D9E04D05B50442E8B781654D,Merged Data AOI,http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxPolylineSet/01275F72D9E04D05B50442E8B781654D


The script below iterates through the polygon dataframe, calls the polylineset url, calls the object url, checks if the folder path from the objects url is equal to the live trace outline path. If it is, it creates a polyline from that. If not, it skips it. Script works, but I need to manipulate the name of the polygon because the polygon names have invalid characters that Arc does not like. "#" as leading character, "-","()","."," " etc...

In [5]:
livetrace = pd.DataFrame()
startTime = datetime.now()
for index,record in df_poly.iterrows():
    dbxpolylineset_url = requests.get(record[2], auth=HttpNegotiateAuth())
    dbxobject_url = requests.get(dbxpolylineset_url.json()["dbxObjectURL"],auth=HttpNegotiateAuth())
    polygonname = dbxpolylineset_url.json()["Name"]
    print(f"Requesting json objects for {polygonname},completed {index}.")
    if dbxobject_url.json()["FolderPath"] == "\Project\Input\Reference Datasets\Live Trace Outlines\Guyana":
        shapeurl = requests.get(dbxobject_url.json()["ShapeURL"],auth=HttpNegotiateAuth())
        livetrace = livetrace.append(record,ignore_index=True)
        print(f"{polygonname} has been added to live trace.")
        dfshape = pd.json_normalize(shapeurl.json()["Geometry"]["LineStrings3"])
        listdict_df = pd.DataFrame()
        polygonname = polygonname.replace("-","_")
        polygonname = polygonname.replace(" ","_")
        polygonname = polygonname.replace("(","")
        polygonname = polygonname.replace(")","")
        
        for index,row in dfshape.iterrows():
            listdict_df2 = pd.json_normalize(dfshape.iloc[index,0])
            listdict_df2["Index"] = index
            listdict_df = listdict_df.append(listdict_df2)
        
        sdf = pd.DataFrame.spatial.from_xy(df=listdict_df,x_column = "x",y_column="y",sr=4326)
        display(sdf.head())

        point = sdf.spatial.to_featureclass(location=f"C:\\Users\\srorta\\Desktop\\Guyana\\MyProject\\MyProject.gdb\\f_{polygonname}",overwrite=True)
        linepath = f"C:\\Users\\srorta\\Desktop\\Guyana\\MyProject\\MyProject.gdb\\f_{polygonname}_line"
        arcpy.management.PointsToLine(point,linepath,"Index")
print("\n")        
print(f"Filter has been completed. ")
print(datetime.now()-startTime)

Requesting json objects for at_bsclos_non,completed 0.
Requesting json objects for D_NTG_Low Quality,completed 1.
Requesting json objects for Mesquite-441,completed 2.
Requesting json objects for Carbonate Slide Block Lead 3,completed 3.
Requesting json objects for LIZ_1P3,completed 4.
Requesting json objects for D_NTG_Medium Quality_1.4,completed 5.
Requesting json objects for P2 Clinoforms - 1,completed 6.
Requesting json objects for C1 Central,completed 7.
Requesting json objects for at_salt_outline,completed 8.
Requesting json objects for Merged Data AOI,completed 9.
Requesting json objects for Striped,completed 10.
Requesting json objects for at_g_platf_y1,completed 11.
Requesting json objects for E East 2,completed 12.
Requesting json objects for Clam,completed 13.
Requesting json objects for WEASPAN_v30_Ghana_(90_Ma),completed 14.
Requesting json objects for C2 Main-3,completed 15.
Requesting json objects for  M_Paleo_HAC,completed 16.
Requesting json objects for Liza_5_WAVSP Or

,x,y,z,measure,Index,SHAPE
0,-56.921542,7.940435,0.0,None,0,"{'x': -56.921542126807296, 'y': 7.94043540146646, 'spatialReference': {'wkid': 4326}}"
1,-56.805296,8.171283,0.0,None,0,"{'x': -56.80529570058119, 'y': 8.171283087763289, 'spatialReference': {'wkid': 4326}}"
2,-56.987653,8.262519,0.0,None,0,"{'x': -56.9876529712664, 'y': 8.262519272169149, 'spatialReference': {'wkid': 4326}}"
3,-56.987766,8.262519,0.0,None,0,"{'x': -56.9877664760026, 'y': 8.262519275654629, 'spatialReference': {'wkid': 4326}}"
4,-56.987880,8.262350,0.0,None,0,"{'x': -56.9878799859144, 'y': 8.2623496771575, 'spatialReference': {'wkid': 4326}}"


Requesting json objects for E_ModelPolygons,completed 126.
Requesting json objects for Polygon 3 East,completed 127.
Requesting json objects for Res 4 Clip OWC -5424m TVDss,completed 128.
Requesting json objects for marl_carbonate,completed 129.
Requesting json objects for C2 East-1,completed 130.
Requesting json objects for Cretaceous Channel 54,completed 131.
Requesting json objects for base_Paleocene shelf,completed 132.
Requesting json objects for 1G2,completed 133.
Requesting json objects for Cretaceous Channel 47,completed 134.
Requesting json objects for Copy (2) of Pal_21,completed 135.
Requesting json objects for E_Mud-prone Channel Complex_15,completed 136.
Requesting json objects for E West A,completed 137.
Requesting json objects for Copy of Potoco Internal 1 Boundary Clipped at Block Boundary,completed 138.
Requesting json objects for Voltzberg_North_P10,completed 139.
Requesting json objects for Cretaceous Channel 67,completed 140.
Requesting json objects for C2 East,comp

,x,y,z,measure,Index,SHAPE
0,-57.501949,8.354248,0.0,None,0,"{'x': -57.5019493254707, 'y': 8.354247732228929, 'spatialReference': {'wkid': 4326}}"
1,-57.501722,8.354361,0.0,None,0,"{'x': -57.5017224164341, 'y': 8.3543610830579, 'spatialReference': {'wkid': 4326}}"
2,-57.501723,8.354700,0.0,None,0,"{'x': -57.5017228497767, 'y': 8.35470027223014, 'spatialReference': {'wkid': 4326}}"
3,-57.501496,8.354814,0.0,None,0,"{'x': -57.5014959404133, 'y': 8.35481362294607, 'spatialReference': {'wkid': 4326}}"
4,-57.501496,8.355153,0.0,None,0,"{'x': -57.5014963735841, 'y': 8.355152812122391, 'spatialReference': {'wkid': 4326}}"


Requesting json objects for Copy of Busymouth,completed 156.
Requesting json objects for E Producers,completed 157.
Requesting json objects for Potoco Interval 5 Boundary Clipped at Block Boundary,completed 158.
Requesting json objects for Turonian or CenFS Source East Poly,completed 159.
Requesting json objects for D_NTG_Low Quality_1.6,completed 160.
Requesting json objects for Clip_Polygon,completed 161.
Requesting json objects for Foureye,completed 162.
Requesting json objects for 2017_Guyana_Basin_Reg_Velocity_Model_2017_Regional_Velocity_Model_VAvg_T,completed 163.
2017_Guyana_Basin_Reg_Velocity_Model_2017_Regional_Velocity_Model_VAvg_T has been added to live trace.


...

Requesting json objects for Potoco 3 Depth Adjustment Polygons,completed 1757.
Requesting json objects for GY_CGG17_4DBL_FWI_2018CGG_4x4_aeps_tti_d,completed 1758.
GY_CGG17_4DBL_FWI_2018CGG_4x4_aeps_tti_d has been added to live trace.


,x,y,z,measure,Index,SHAPE
0,-57.066800,7.878294,0.0,None,0,"{'x': -57.0667995101959, 'y': 7.87829388877631, 'spatialReference': {'wkid': 4326}}"
1,-56.833881,7.878266,0.0,None,0,"{'x': -56.833881117472, 'y': 7.87826629766651, 'spatialReference': {'wkid': 4326}}"
2,-56.833691,8.340942,0.0,None,0,"{'x': -56.83369111621109, 'y': 8.340941938741729, 'spatialReference': {'wkid': 4326}}"
3,-57.067103,8.340971,0.0,None,0,"{'x': -57.0671029680366, 'y': 8.34097113346536, 'spatialReference': {'wkid': 4326}}"
4,-57.067026,7.878294,0.0,None,0,"{'x': -57.0670263052144, 'y': 7.8782938525766095, 'spatialReference': {'wkid': 4326}}"


Requesting json objects for at_S_Namib,completed 1759.
Requesting json objects for P1 - Carbonate Buildup Lead 12,completed 1760.
Requesting json objects for Payara Upper West,completed 1761.
Requesting json objects for E South_Oil leg,completed 1762.
Requesting json objects for Cretaceous Channel 80,completed 1763.
Requesting json objects for D Dim,completed 1764.
Requesting json objects for D_NTG_Low Quality_2.3,completed 1765.
Requesting json objects for PART24,completed 1766.
Requesting json objects for E Downdip,completed 1767.
Requesting json objects for Seismic_3D_Priority2A,completed 1768.
Requesting json objects for FrGuiana_Blks_Jan2017,completed 1769.
Requesting json objects for LIZ_4i11_GSTGE_MEM_20210322_TOP_C2.dat,completed 1770.
Requesting json objects for E Other East,completed 1771.
Requesting json objects for Cretaceous Channel 63,completed 1772.
Requesting json objects for Leaffish,completed 1773.
Requesting json objects for Top1_Poly,completed 1774.
Requesting json 

,x,y,z,measure,Index,SHAPE
0,-57.502452,8.355533,0.0,None,0,"{'x': -57.502452448373795, 'y': 8.355532577985441, 'spatialReference': {'wkid': 4326}}"
1,-57.501545,8.356438,0.0,None,0,"{'x': -57.501545386806, 'y': 8.356438234114329, 'spatialReference': {'wkid': 4326}}"
2,-57.501547,8.357343,0.0,None,0,"{'x': -57.5015465423104, 'y': 8.35734273857204, 'spatialReference': {'wkid': 4326}}"
3,-57.500639,8.358248,0.0,None,0,"{'x': -57.5006394745954, 'y': 8.35824839291327, 'spatialReference': {'wkid': 4326}}"
4,-57.500641,8.359153,0.0,None,0,"{'x': -57.5006406282661, 'y': 8.35915289741433, 'spatialReference': {'wkid': 4326}}"


Requesting json objects for Barracuda-Piranha,completed 1784.
Requesting json objects for Polygon 5,completed 1785.
Requesting json objects for Channel Complex 3_NTG,completed 1786.
Requesting json objects for Tacutu,completed 1787.
Requesting json objects for Millet,completed 1788.
Requesting json objects for Canje Map Boundary,completed 1789.
Requesting json objects for Base of Western fairway_5_perimeter,completed 1790.
Requesting json objects for Polygons 15,completed 1791.
Requesting json objects for jja_data limit Pre S,completed 1792.
Requesting json objects for LIZ_2P4_GSTGE_MEM_20200611_TOP_D.dat,completed 1793.
Requesting json objects for Final L5 Flex Ma2_150 Top F-5330,completed 1794.
Requesting json objects for C1 Central-4,completed 1795.
Requesting json objects for LIZ_4P24,completed 1796.
Requesting json objects for Snoek_Lead,completed 1797.
Requesting json objects for Landed D Water Injector Completion Sections,completed 1798.
Requesting json objects for AB2 Baffle 3,

Creates separate dataframe of polygons within the guyana live trace outline folder. 

In [89]:
livetrace = pd.DataFrame()
startTime = datetime.now()
for index,record in df_poly.iterrows():
    obj_f = f"{base_url}/api/studio/petrel/objects/{connection}/{repository}/{polygons}/{record[0]}"
    obj_f_req = requests.get(obj_f, auth=HttpNegotiateAuth())
    name = obj_f_req.json()["properties"]["Name"]
    print(f"Requesting json objects for {name},completed {index}.")
    if obj_f_req.json()["properties"]["FolderPath"] == "\Project\Input\Reference Datasets\Live Trace Outlines\Guyana":
        livetrace = livetrace.append(record,ignore_index=True)
        
        print(f"{name} has been added to live trace.")

print("\n")        
print(f"Filter has been completed. ")
print(datetime.now()-startTime)

Requesting json objects for at_bsclos_non,completed 0.
Requesting json objects for D_NTG_Low Quality,completed 1.
Requesting json objects for Mesquite-441,completed 2.
Requesting json objects for Carbonate Slide Block Lead 3,completed 3.
Requesting json objects for LIZ_1P3,completed 4.
Requesting json objects for D_NTG_Medium Quality_1.4,completed 5.
Requesting json objects for P2 Clinoforms - 1,completed 6.
Requesting json objects for C1 Central,completed 7.
Requesting json objects for at_salt_outline,completed 8.
Requesting json objects for Merged Data AOI,completed 9.
Requesting json objects for Striped,completed 10.
Requesting json objects for at_g_platf_y1,completed 11.
Requesting json objects for E East 2,completed 12.
Requesting json objects for Clam,completed 13.
Requesting json objects for WEASPAN_v30_Ghana_(90_Ma),completed 14.
Requesting json objects for C2 Main-3,completed 15.
Requesting json objects for  M_Paleo_HAC,completed 16.
Requesting json objects for Liza_5_WAVSP Or

Requesting json objects for M_Paleo_ErRem_HAC,completed 254.
Requesting json objects for at_g_fan_u,completed 255.
Requesting json objects for Cretaceous Channel 8,completed 256.
Requesting json objects for Incision into Top Carbonate,completed 257.
Requesting json objects for Cretaceous Channel 42,completed 258.
Requesting json objects for Hammerhead,completed 259.
Requesting json objects for F Other,completed 260.
Requesting json objects for aa_anti,completed 261.
Requesting json objects for sr_blk52_3d,completed 262.
Requesting json objects for D Dim,completed 263.
Requesting json objects for South Caribbean Accret.Prism,completed 264.
Requesting json objects for E Downdip,completed 265.
Requesting json objects for jja_graphita_lo_mod_,completed 266.
Requesting json objects for Karst Lead 1 Area D-A-T Calc,completed 267.
Requesting json objects for Copy of Croaker Perched Fan,completed 268.
Requesting json objects for Copy of Merged_sorubim,completed 269.
Requesting json objects for

Requesting json objects for Croaker,completed 507.
Requesting json objects for Canje_1_Mask,completed 508.
Requesting json objects for LIZ_4P24_GSTGE_MEM_20210813_TOP_C1.dat,completed 509.
Requesting json objects for Polygon 1,completed 510.
Requesting json objects for Cretaceous Channel 36 (Liza),completed 511.
Requesting json objects for P4 - Carbonate Buildup Lead 3,completed 512.
Requesting json objects for Guppy,completed 513.
Requesting json objects for Payara Lower South,completed 514.
Requesting json objects for at_a_5000,completed 515.
Requesting json objects for E North,completed 516.
Requesting json objects for AB1_ChannelMargin,completed 517.
Requesting json objects for E Dim South,completed 518.
Requesting json objects for Liza 5 - Payara 2 MTC Geobody Extent,completed 519.
Requesting json objects for Liza Deep,completed 520.
Requesting json objects for 1P2,completed 521.
Requesting json objects for Copy of TOP LIZA C-5346,completed 522.
Requesting json objects for E Dim E

Requesting json objects for Cretaceous Channel 34,completed 754.
Requesting json objects for D_NTG_Medium Quality_1.3,completed 755.
Requesting json objects for aajja_amatista_dwnle,completed 756.
Requesting json objects for E_Mud-prone Channel Complex_5,completed 757.
Requesting json objects for Polygon Campanian Clean Seal  1,completed 758.
Requesting json objects for F main,completed 759.
Requesting json objects for P5 Carbonate Buildup 20,completed 760.
Requesting json objects for Bemau_G_Lower_P90,completed 761.
Requesting json objects for Top Potoco Depth Adjustment Polygons,completed 762.
Requesting json objects for P3 Clinoforms - 1,completed 763.
Requesting json objects for L_Paleo_Green,completed 764.
Requesting json objects for Aimara Ext. SW,completed 765.
Requesting json objects for Copy of Maastrichtian Channel Complex 5 composed of channels 48, 49,completed 766.
Requesting json objects for midpaleo_blue,completed 767.
Requesting json objects for E_Mud-prone Channel Compl

Requesting json objects for at_sfM_carb,completed 999.
Requesting json objects for C2_PLC_1,completed 1000.
Requesting json objects for D_NTG_Medium Quality_3.1,completed 1001.
Requesting json objects for M_Paleo_HAD,completed 1002.
Requesting json objects for GY_WT_TBFWI-TCE4DBL_Merge_EM2021_2x2,completed 1003.
GY_WT_TBFWI-TCE4DBL_Merge_EM2021_2x2 has been added to live trace.
Requesting json objects for F Final NDC Flex Oil Leg Boundary,completed 1004.
Requesting json objects for D Injector 1k Radius,completed 1005.
Requesting json objects for Res D Outline,completed 1006.
Requesting json objects for C1_ErodedChannel,completed 1007.
Requesting json objects for Down-Dip Eagle,completed 1008.
Requesting json objects for 13 - Ma2_300 (C1 west east 3),completed 1009.
Requesting json objects for at_bsclos_disc,completed 1010.
Requesting json objects for Polygon 8,completed 1011.
Requesting json objects for Canje 2 Blanking Polygon,completed 1012.
Requesting json objects for Cretaceous Cha

Requesting json objects for Up_Sorubim_4,completed 1247.
Requesting json objects for Lead_2b,completed 1248.
Requesting json objects for AB2_ntg,completed 1249.
Requesting json objects for Polygons 22,completed 1250.
Requesting json objects for Cretaceous Channel 46,completed 1251.
Requesting json objects for D_NTG_Medium Quality_1.1,completed 1252.
Requesting json objects for gy_kP1_FT_cgg17_1x2_100_KP1_Raw_fl03_36_sshape2db_sec_IMP_diffusion_z,completed 1253.
gy_kP1_FT_cgg17_1x2_100_KP1_Raw_fl03_36_sshape2db_sec_IMP_diffusion_z has been added to live trace.
Requesting json objects for blk52_53_3d_XOMQ,completed 1254.
Requesting json objects for Paleogene (Zone 3),completed 1255.
Requesting json objects for Liza GPF Outline',completed 1256.
Requesting json objects for D_NTG_Low Quality_3.2,completed 1257.
Requesting json objects for C2_Oil5345,completed 1258.
Requesting json objects for F_ConfinedChannel,completed 1259.
Requesting json objects for KU_Lead_4CH,completed 1260.
Requestin

Requesting json objects for LIZ_2W6_GSTGE_MEM_20201014_TOP_D.dat,completed 1486.
Requesting json objects for E_Mud-prone Channel Complex_1,completed 1487.
Requesting json objects for -5273 m TVDSS Payara Res 5 OWC,completed 1488.
Requesting json objects for Maastrichtian Channel Complex 2 composed of channels 7, 9, 15, 20, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 35,completed 1489.
Requesting json objects for D_NTG_Low Quality_2.1,completed 1490.
Requesting json objects for C0_ChannelMargin,completed 1491.
Requesting json objects for lead_,completed 1492.
Requesting json objects for sr_block31_tgs10_1x15,completed 1493.
Requesting json objects for Kuma Kuma,completed 1494.
Requesting json objects for Purpleheart,completed 1495.
Requesting json objects for Cuiu-Cuiu,completed 1496.
Requesting json objects for at_inflex,completed 1497.
Requesting json objects for stbr_srbm_cgg15_1x1,completed 1498.
stbr_srbm_cgg15_1x1 has been added to live trace.
Requesting json objects for Payar

Requesting json objects for C WEST_North,completed 1727.
Requesting json objects for Polygons 2,completed 1728.
Requesting json objects for Up_Sorubim_1,completed 1729.
Requesting json objects for LIZ_4P21_GSTGE_MEM_20210715_TOP_E.dat,completed 1730.
Requesting json objects for Liza 1 Polygon,completed 1731.
Requesting json objects for Payara Lower South,completed 1732.
Requesting json objects for Flathead,completed 1733.
Requesting json objects for Cretaceous Channel 37,completed 1734.
Requesting json objects for F Main_NoWedge_clipped -5330,completed 1735.
Requesting json objects for Stabroek_Canje_3D_Waterbottom,completed 1736.
Requesting json objects for Polygons 13,completed 1737.
Requesting json objects for LIZ_4P21,completed 1738.
Requesting json objects for D_NTG_High Quality_3,completed 1739.
Requesting json objects for Poly_Base6_8,completed 1740.
Requesting json objects for Liza 01 BP02_WAVSP Geometry,completed 1741.
Requesting json objects for D_Channel Complex 4,completed 

In [34]:
#So you need to first call polylineset table to get list of guids for each polygon. Then you call the dbxobject URL from that response to get shape url, then you call the shape url to get geometries. 
polygon_url_guid = f"{base_url}/api/studio/petrel/entities/{connection}/{repository}/{entityType}/2DD0EA51C868458D93B8121D5A04E366"
f = polygon_req = requests.get(polygon_url_guid, auth=HttpNegotiateAuth())
df_f = pd.json_normalize(f.json())

objectshape = requests.get(f.json()["dbxObjectURL"],auth=HttpNegotiateAuth())
objectshape_df = pd.json_normalize(objectshape.json())

objectshape.json()["ShapeURL"]
shapegeom = requests.get(objectshape.json()["ShapeURL"],auth=HttpNegotiateAuth())
shapegeom_df = pd.json_normalize(shapegeom.json())
name = f.json()["Name"]
display(df_f.transpose())
#display(objectshape_df.transpose())
#display(shapegeom_df.transpose())

,0
Id,575916
Guid,2DD0EA51C868458D93B8121D5A04E366
Name,Falcon
JsonDoc,None
JsonAdminDoc,None
SRD,NaN
SRDName,None
SRDVelocity,None
SRDTimeShift,None
Comments,None


In [37]:
shapegeom.json()

{'Geometry': {'Points3': [{'x': -69.41976560719701, 'y': 11.7671586482532, 'z': 0.0, 'measure': None}, {'x': -69.36091411459469, 'y': 11.7199578538724, 'z': 0.0, 'measure': None}, {'x': -69.2797703431268, 'y': 11.6608465471302, 'z': 0.0, 'measure': None}, {'x': -69.2127385326571, 'y': 11.5982442080038, 'z': 0.0, 'measure': None}, {'x': -69.12806677116448, 'y': 11.5391067166018, 'z': 0.0, 'measure': None}, {'x': -69.0328110386984, 'y': 11.5043140573479, 'z': 0.0, 'measure': None}, {'x': -68.9340273171069, 'y': 11.4660370989333, 'z': 0.0, 'measure': None}, {'x': -68.8281876145667, 'y': 11.4451565434396, 'z': 0.0, 'measure': None}, {'x': -68.7258759029505, 'y': 11.4103521498318, 'z': 0.0, 'measure': None}, {'x': -68.6094522312354, 'y': 11.3685811732826, 'z': 0.0, 'measure': None}, {'x': -68.4967147142126, 'y': 11.3209575050848, 'z': 0.0, 'measure': None}, {'x': -68.4960930137815, 'y': 11.320575738379599, 'z': 0.0, 'measure': None}, {'x': -68.44346210299359, 'y': 11.2882548360919, 'z': 0.0

what does some polygons not have the ["LineStrings3"] key?

In [30]:
dfshape = pd.json_normalize(shapegeom.json()["Geometry"])
dfshape

listdict_df = pd.DataFrame()
for index,row in dfshape.iterrows():
    listdict_df2 = pd.json_normalize(dfshape.iloc[index,0])
    listdict_df2["Index"] = index
    listdict_df = listdict_df.append(listdict_df2)
display(listdict_df)

,x,y,z,measure,Index
0,-58.733388,9.503970,2.500000,None,0
1,-58.727361,9.477790,0.000000,None,0
2,-58.718850,9.460107,2.500000,None,0
3,-58.708769,9.443790,0.000000,None,0
4,-58.700933,9.436847,2.500000,None,0
5,-58.690312,9.436812,0.000000,None,0
6,-58.677346,9.439784,0.000000,None,0
7,-58.665234,9.446078,0.000000,None,0
8,-58.654755,9.447736,2.500000,None,0
9,-58.645553,9.445755,0.000000,None,0


In [31]:
from arcgis.features import GeoAccessor,GeoSeriesAccessor
sdf = pd.DataFrame.spatial.from_xy(df=listdict_df,x_column = "x",y_column="y",sr=4326)
display(sdf.head())

sdf.spatial.to_featureclass(location=f"C:\\Users\\srorta\\Desktop\\Guyana\\MyProject\\MyProject.gdb\\{name}",overwrite=True)

arcpy.management.PointsToLine(r"C:\Users\srorta\Desktop\Guyana\MyProject\MyProject.gdb\polygon_studio",f"C:\\Users\\srorta\\Desktop\\Guyana\\MyProject\\MyProject.gdb\\polygon_studio_line","Index")

arcpy.management.FeatureToPolygon("polygon_studio_line",r"C:\Users\srorta\Desktop\Guyana\MyProject\MyProject.gdb\polygon_studio_polygon")

,x,y,z,measure,Index,SHAPE
0,-58.733388,9.503970,2.5,None,0,"{'x': -58.7333883582955, 'y': 9.50397002553396, 'spatialReference': {'wkid': 4326}}"
1,-58.727361,9.477790,0.0,None,0,"{'x': -58.727361486319396, 'y': 9.47778971734097, 'spatialReference': {'wkid': 4326}}"
2,-58.718850,9.460107,2.5,None,0,"{'x': -58.718849667932, 'y': 9.46010687980122, 'spatialReference': {'wkid': 4326}}"
3,-58.708769,9.443790,0.0,None,0,"{'x': -58.7087688377912, 'y': 9.44379041963033, 'spatialReference': {'wkid': 4326}}"
4,-58.700933,9.436847,2.5,None,0,"{'x': -58.70093274061939, 'y': 9.43684743778948, 'spatialReference': {'wkid': 4326}}"


ExecuteError: ERROR 000354: The name contains invalid characters
Failed to execute (CreateFeatureclass).
